In [6]:
import numpy as np
import sympy as sp

In [11]:
"""Symbols"""
theta1, theta2, theta1_dot, theta2_dot, theta1_ddot, theta2_ddot = sp.symbols('theta1 theta2 theta1_dot theta2_dot, theta1_ddot, theta2_ddot')

"""Parameters"""
m1, m2, l1, l2 = 1, 1, 1, 1
g = 9.81
y0 = np.array([np.pi/2, np.pi/2, 0, 0])

"""Equations"""
eq1 = (m1 + m2) * l1 * theta1_ddot + m2 * l2 * theta2_ddot * sp.cos(theta1 - theta2) + m2 * l2 * theta2_dot**2 * sp.sin(theta1 - theta2) + (m1 + m2) * 9.81 * sp.sin(theta1)
eq2 = l1 * theta1_ddot * sp.cos(theta1 - theta2) + l2 * theta2_ddot - l1 * theta1_dot**2 * sp.sin(theta1 - theta2) + 9.81 * sp.sin(theta2)


"""Resolution of the system"""
sys = sp.solve((eq1,eq2),(theta1_ddot,theta2_ddot), dict = True)[0]
theta1_ddot_ = sp.simplify(sys[theta1_ddot])
theta2_ddot_ = sp.simplify(sys[theta2_ddot])

"""Substitution"""
theta1_ddot_lambdify = sp.lambdify((theta1,theta1_dot,theta2,theta2_dot), theta1_ddot, 'numpy')
theta2_ddot_lambdify = sp.lambdify((theta1,theta1_dot,theta2,theta2_dot), theta2_ddot, 'numpy')

(50.0*theta1_dot**2*sin(2*theta1 - 2*theta2) + 100.0*theta2_dot**2*sin(theta1 - theta2) + 1471.5*sin(theta1) + 490.5*sin(theta1 - 2*theta2))/(100.0*cos(theta1 - theta2)**2 - 200.0)
(-100.0*theta1_dot**2*sin(theta1 - theta2) - 25.0*theta2_dot**2*sin(2*theta1 - 2*theta2) + 490.5*sin(theta2) - 490.5*sin(2*theta1 - theta2))/(50.0*cos(theta1 - theta2)**2 - 100.0)


In [13]:
def function(t, y):
    return np.array([y[2], y[3], theta1_ddot_lambdify(y[0], y[1], y[2], y[3]), theta2_ddot_lambdify(y[0], y[1], y[2], y[3])])

## Modélisation physique du pendule double